# Customize State

- Add additional fields to the state
- The chatbot can access tools and forward the results to a human for review.
- This requires a relatively large LLM.

In [1]:
# Built-in library
import asyncio
import json
import logging
import re
import warnings
from pathlib import Path
from pprint import pprint
from typing import (
    Annotated,
    Any,
    Generator,
    Iterable,
    Literal,
    Optional,
    TypedDict,
    Union,
)

# Standard imports
import nest_asyncio
import numpy as np
import numpy.typing as npt
import pandas as pd
import polars as pl
from rich.console import Console
from rich.theme import Theme

custom_theme = Theme(
    {
        "white": "#FFFFFF",  # Bright white
        "info": "#00FF00",  # Bright green
        "warning": "#FFD700",  # Bright gold
        "error": "#FF1493",  # Deep pink
        "success": "#00FFFF",  # Cyan
        "highlight": "#FF4500",  # Orange-red
    }
)
console = Console(theme=custom_theme)

# Visualization
# import matplotlib.pyplot as pltife

# NumPy settings
np.set_printoptions(precision=4)

# Pandas settings
pd.options.display.max_rows = 1_000
pd.options.display.max_columns = 1_000
pd.options.display.max_colwidth = 600

# Polars settings
pl.Config.set_fmt_str_lengths(1_000)
pl.Config.set_tbl_cols(n=1_000)

warnings.filterwarnings("ignore")

# Black code formatter (Optional)
%load_ext lab_black

# auto reload imports
%load_ext autoreload
%autoreload 2

In [2]:
def go_up_from_current_directory(*, go_up: int = 1) -> None:
    """This is used to up a number of directories.

    Params:
    -------
    go_up: int, default=1
        This indicates the number of times to go back up from the current directory.

    Returns:
    --------
    None
    """
    import os
    import sys

    CONST: str = "../"
    NUM: str = CONST * go_up

    # Goto the previous directory
    prev_directory = os.path.join(os.path.dirname(__name__), NUM)
    # Get the 'absolute path' of the previous directory
    abs_path_prev_directory = os.path.abspath(prev_directory)

    # Add the path to the System paths
    sys.path.insert(0, abs_path_prev_directory)
    print(abs_path_prev_directory)

In [3]:
go_up_from_current_directory(go_up=2)


from schemas import ModelEnum  # noqa: E402
from settings import refresh_settings  # noqa: E402
from utilities.client_utils import check_rate_limit  # noqa: E402

settings = refresh_settings()

/Users/neidu/Desktop/Projects/Personal/My_Projects/AI-Tutorials


In [4]:
from langgraph.graph.message import add_messages


class State(TypedDict):
    messages: Annotated[list, add_messages]
    name: str
    birthday: str

### Update The States Inside The Tool

In [ ]:
from IPython.display import Image, display
from langchain.chat_models import init_chat_model
from langchain_core.messages import ToolMessage
from langchain_core.tools import InjectedToolCallId, tool
from langchain_tavily import TavilySearch
from langfuse.callback import CallbackHandler
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import END, START, StateGraph
from langchain_litellm import ChatLiteLLM
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.types import Command, interrupt


# Note: Because we're generating a ToolMessage for a state update, we generally require
# the ID of the corresponding too call. We can use LangChain's InjectedToolCallId to
# signal that this argument should not be revealed to the model in the tool's schema.
@tool
def human_assistance(
    name: str, birthday: str, tool_call_id: Annotated[str, InjectedToolCallId]
) -> str:
    """Request assistance from a human."""
    human_response = interrupt(
        {
            "question": "Is this correct?",
            "name": name,
            "birthday": birthday,
        }
    )
    # If the info is correct, update the state
    if human_response.get("correct", "").lower().startswith("y"):
        verified_name = name
        verified_birthday = birthday
        response = "Correct"

    else:
        # Get the corrected information from the human
        verified_name = human_response.get("name", "")
        verified_birthday = human_response.get("birthday", "")
        response = f"Made a correction: {human_response}"
    # Explicitly update the state with a ToolMessage
    state_update = {
        "name": verified_name,
        "birthday": verified_birthday,
        "messages": [ToolMessage(response, tool_call_id=tool_call_id)],
    }
    # Return a Command object
    return Command(update=state_update)

In [ ]:
# from langchain_litellm import ChatLiteLLM

# LLM = ChatLiteLLM(
#     model=f"openrouter/{ModelEnum.QWEN_3p0_32B_REMOTE_FREE.value}",
#     temperature=0.1,
#     openrouter_api_key=settings.OPENROUTER_API_KEY.get_secret_value(),
# )

# LLM.invoke("Who are you?").content

"Hello! I am Qwen, a large-scale language model developed by Alibaba Cloud's Tongyi Lab. I can assist you with a wide range of tasks, including answering questions, creating text (like writing stories, official documents, emails, and scripts), performing logical reasoning, coding, and more. I support multiple languages and am designed to be helpful, harmless, and honest. How can I assist you today?"

In [ ]:
async def chatbot(state: State) -> dict[str, Any]:
    """Process chat messages through LLM with tools and return response.

    Parameters
    ----------
    state : State
        Current state containing message history.

    Returns
    -------
    dict[str, Any]
        Dictionary containing LLM response message.
        Contains key 'messages' with list of one message.

    Notes
    -----
    Disables parallel tool calling to prevent duplicate tool invocations
    when restarting the graph flow. Asserts at most one tool call per message.
    """
    message = await llm_with_tools.ainvoke(state["messages"])
    # Disable parallel tool calling because we'll be interrupting (human-in-the-loop)
    # to prevent repeating any tool invocations when we restart the graph
    assert len(message.tool_calls) <= 1
    return {"messages": [message]}


# A simple memory saver for this tutorial. In production,
# it's recommennded to use SqliteSaver or PostgresSaver
memory = MemorySaver()

model_str: str = "mistralai:mistral-large-latest"  # "mistralai:ministral-8b-latest"
llm = init_chat_model(model_str)
tavily_search = TavilySearch(max_results=2)
tools = [tavily_search, human_assistance]
llm_with_tools = llm.bind_tools(tools)

# Add nodes
graph_builder = StateGraph(State)
tool_node = ToolNode(tools=tools)
graph_builder.add_node("chatbot", chatbot)
graph_builder.add_node("tools", tool_node)

# Connect nodes
graph_builder.add_edge(START, "chatbot")
graph_builder.add_conditional_edges("chatbot", tools_condition)
graph_builder.add_edge("tools", "chatbot")
graph_builder.add_edge("chatbot", END)

# Add memory, observability and compile the graph
memory = MemorySaver()
langfuse_handler = CallbackHandler()
graph = graph_builder.compile(checkpointer=memory).with_config({"callbacks": [langfuse_handler]})

### Prompt The Chatbot

- Prompt the chatbot to lookup when LangGraph was created (`birthday`) and direct the chatbot to reach out to the human_assistance tool once it has the reqired info.
- By setting the `name` and `birthday` in the arguments for the tool, we force the chatbot to generate proposals for these fields.

In [10]:
user_input: str = (
    "Can you look up when LangGraph was released? When you "
    "have the answer, use the human_assistance tool for review."
)
config = {"configurable": {"thread_id": "1"}}

events = graph.astream(
    {"messages": [{"role": "user", "content": user_input}]},
    config=config,
    stream_mode="values",
)
async for event in events:
    if "messages" in event:
        event["messages"][-1].pretty_print()

================================ Human Message =================================

Can you look up when LangGraph was released? When you have the answer, use the human_assistance tool for review.
================================== Ai Message ==================================
Tool Calls:
  tavily_search (4W56yVM4L)
 Call ID: 4W56yVM4L
  Args:
    query: LangGraph release date
================================= Tool Message =================================
Name: tavily_search

{"query": "LangGraph release date", "follow_up_questions": null, "answer": null, "images": [], "results": [{"title": "Announcing LangGraph v0.1 & LangGraph Cloud: Running agents at scale ...", "url": "https://blog.langchain.dev/langgraph-cloud/", "content": "Announcing LangGraph v0.1 & LangGraph Cloud: Running agents at scale, reliably Our new infrastructure for running agents at scale, LangGraph Cloud, is available in beta. Separate from the langchain package, LangGraph’s core design philosophy is to help develope

### Add Human Assistance

- The chatbot failed to identify the correct date.
- We'll supply the correct information to it.

In [11]:
human_command = Command(
    resume={
        "name": "LangGraph",
        "birthday": "Jan 17, 2024",
    }
)

events = graph.astream(human_command, config, stream_mode="values")
async for event in events:
    if "messages" in event:
        event["messages"][-1].pretty_print()

================================== Ai Message ==================================
Tool Calls:
  human_assistance (FUCAd4Bf5)
 Call ID: FUCAd4Bf5
  Args:
    name: Joshua
    birthday: 1990-01-01
================================= Tool Message =================================
Name: human_assistance

Made a correction: {'name': 'LangGraph', 'birthday': 'Jan 17, 2024'}
================================== Ai Message ==================================

LangGraph was released on January 17, 2024.


In [12]:
# The state has been updated to reflect the human's input
snapshot = graph.get_state(config)
{k: v for k, v in snapshot.values.items() if k in ("name", "birthday")}

{'name': 'LangGraph', 'birthday': 'Jan 17, 2024'}

## Manually Update The State

In [13]:
graph.update_state(config, {"name": "LangGraph (library)"})

{'configurable': {'thread_id': '1',
  'checkpoint_ns': '',
  'checkpoint_id': '1f038a71-4a83-6d0c-8006-c92bc4c6881b'}}

### View The New State

In [14]:
snapshot = graph.get_state(config)
{k: v for k, v in snapshot.values.items() if k in ("name", "birthday")}

{'name': 'LangGraph (library)', 'birthday': 'Jan 17, 2024'}

<br>

## Time Travel

- Chatbot workflows typically involve users interacting with a bot to complete a task.
- Memory and human-in-the-loop features help manage the bot's state and responses.
- LangGraph's time travel functionality allows users to restart from previous points, explore different outcomes, or correct mistakes.

- This is a continuation from the previous section (customizing the state).

In [ ]:
memory = MemorySaver()

model_str: str = "mistralai:mistral-large-latest"  # "mistralai:ministral-8b-latest"
llm = init_chat_model(model_str)
tavily_search = TavilySearch(max_results=2)
tools = [tavily_search, human_assistance]
llm_with_tools = llm.bind_tools(tools)

# Add nodes
graph_builder = StateGraph(State)
tool_node = ToolNode(tools=tools)
graph_builder.add_node("chatbot", chatbot)
graph_builder.add_node("tools", tool_node)

# Connect nodes
graph_builder.add_edge(START, "chatbot")
graph_builder.add_conditional_edges("chatbot", tools_condition)
graph_builder.add_edge("tools", "chatbot")
graph_builder.add_edge("chatbot", END)

# Add memory, observability and compile the graph
memory = MemorySaver()
langfuse_handler = CallbackHandler()
graph = graph_builder.compile(checkpointer=memory).with_config({"callbacks": [langfuse_handler]})

### Add Steps

In [ ]:
user_input: str = "I'm learning LangGraph. Could you do some research on it for me?"
config = {"configurable": {"thread_id": "1"}}

events = graph.astream(
    {"messages": [{"role": "user", "content": user_input}]},
    config=config,
    stream_mode="values",
)
async for event in events:
    if "messages" in event:
        # event["messages"][-1].pretty_print()
        console.log(event["messages"][-1])

[15:06:31] HumanMessage(                                                                           ]8;id=473683;file:///var/folders/ny/dl75sc_x2tb54lsymt5bh5p00000gn/T/ipykernel_37674/2202341719.py\2202341719.py]8;;\:]8;id=14563;file:///var/folders/ny/dl75sc_x2tb54lsymt5bh5p00000gn/T/ipykernel_37674/2202341719.py#12\12]8;;\
               content="I'm learning LangGraph. Could you do some research on it for me?",                         
               additional_kwargs={},                                                                               
               response_metadata={},                                                                               
               id='2e79bdee-fb03-43c3-a85e-03dea24cd730'                                                           
           )                                                                                                       

[15:06:34] AIMessage(                                                                              ]8;id=762996;file:///var/folders/ny/dl75sc_x2tb54lsymt5bh5p00000gn/T/ipykernel_37674/2202341719.py\2202341719.py]8;;\:]8;id=316258;file:///var/folders/ny/dl75sc_x2tb54lsymt5bh5p00000gn/T/ipykernel_37674/2202341719.py#12\12]8;;\
               content='',                                                                                         
               additional_kwargs={                                                                                 
                   'tool_calls': [                                                                                 
                       {                                                                                           
                           'id': 'MAoHZg5Fd',                                                                      
                           'function': {                                                                           
                               'name': 'tavily_search',                                                            
                               'arguments': '{"query": "LangGraph", "include_domains": [],                         
           "exclude_domains": [], "search_depth": "advanced", "include_images": false,                             
           "time_range": null, "topic": "general"}'                                                                
                           },                                                                                      
                           'index': 0                                                                              
                       }                                                                                           
                   ]                                                                                               
               },                                                                                                  
               response_metadata={                                                                                 
                   'token_usage': {                                                                                
                       'prompt_tokens': 1223,                                                                      
                       'total_tokens': 1292,                                                                       
                       'completion_tokens': 69                                                                     
                   },                                                                                              
                   'model_name': 'mistral-large-latest',                                                           
                   'model': 'mistral-large-latest',                                                                
                   'finish_reason': 'tool_calls'                                                                   
               },                                                                                                  
               id='run--5fc37fa9-f14c-4199-94d8-c39abca244e0-0',                                                   
               tool_calls=[                                                                                        
                   {                                                                                               
                       'name': 'tavily_search',                                                                    
                       'args': {                                                                                   
                           'query': 'LangGraph',                                                                   
                           'include_domains': [],                    

[15:06:36] ToolMessage(                                                                            ]8;id=512945;file:///var/folders/ny/dl75sc_x2tb54lsymt5bh5p00000gn/T/ipykernel_37674/2202341719.py\2202341719.py]8;;\:]8;id=810410;file:///var/folders/ny/dl75sc_x2tb54lsymt5bh5p00000gn/T/ipykernel_37674/2202341719.py#12\12]8;;\
               content='{"query": "LangGraph", "follow_up_questions": null, "answer": null,                        
           "images": [], "results": [{"url": "https://www.ibm.com/think/topics/langgraph",                         
           "title": "What is LangGraph? - IBM", "content": "LangGraph, created by LangChain, is an                 
           open source AI agent framework designed to build, deploy and manage complex generative                  
           AI agent workflows. It provides a set of tools and libraries that enable users to                       
           create, run and optimize large language models (LLMs) in a scalable and efficient                       
           manner. At its core, LangGraph uses the power of graph-based architectures to model and                 
           manage the intricate relationships between various components of an AI agent workflow.                  
           [...] Agent systems: LangGraph provides a framework for building agent-based systems,                   
           which can be used in applications such as robotics, autonomous vehicles or video                        
           games.\\n\\nLLM applications: By using LangGraph’s capabilities, developers can build                   
           more sophisticated AI models that learn and improve over time. Norwegian Cruise Line                    
           uses LangGraph to compile, construct and refine guest-facing AI solutions. This                         
           capability allows for improved and personalized guest experiences. [...] By using a                     
           graph-based architecture, LangGraph enables users to scale artificial intelligence                      
           workflows without slowing down or sacrificing efficiency. LangGraph uses enhanced                       
           decision-making by modeling complex relationships between nodes, which means it uses AI                 
           agents to analyze their past actions and feedback. In the world of LLMs, this process                   
           is referred to as reflection.", "score": 0.9353998, "raw_content": null}, {"url":                       
           "https://www.langchain.com/langgraph", "title": "LangGraph - LangChain", "content":                     
           "LangGraph is a stateful, orchestration framework that brings added control to agent                    
           workflows. LangGraph Platform is a service for deploying and scaling LangGraph                          
           applications, with an opinionated API for building agent UXs, plus an integrated                        
           developer studio.\\nLangGraph (open source)\\nLangGraph                                                 
           Platform\\nFeatures\\nDescription\\nStateful orchestration framework for agentic                        
           applications\\nScalable infrastructure for deploying LangGraph                                          
           applications\\nSDKs\\nPython and JavaScript\\nPython and JavaScript", "score":                          
           0.921591, "raw_content": null}], "response_time": 1.06}',                                               
               name='tavily_search',                                                                               
               id='465d5457-93bd-43e3-8305-12e92318ec8c',                                                          
               tool_call_id='MAoHZg5Fd'                                                                            
           )                                                         

[15:06:53] AIMessage(                                                                              ]8;id=910820;file:///var/folders/ny/dl75sc_x2tb54lsymt5bh5p00000gn/T/ipykernel_37674/2202341719.py\2202341719.py]8;;\:]8;id=719497;file:///var/folders/ny/dl75sc_x2tb54lsymt5bh5p00000gn/T/ipykernel_37674/2202341719.py#12\12]8;;\
               content=[                                                                                           
                   {                                                                                               
                       'type': 'text',                                                                             
                       'text': 'LangGraph, developed by LangChain, is an open-source AI agent                      
           framework designed to create, deploy, and manage complex generative AI agent workflows.                 
           It offers a suite of tools and libraries that allow users to build, run, and optimize                   
           large language models (LLMs) efficiently and scalably.\n\n### Key Features:\n\n1.                       
           **Graph-Based Architecture**:\n   - **Modeling Relationships**: LangGraph leverages                     
           graph-based architectures to model and manage the intricate relationships between                       
           various components of an AI agent workflow. This enables enhanced decision-making by                    
           analyzing past actions and feedback, a process known as reflection in the context of                    
           LLMs '                                                                                                  
                   },                                                                                              
                   {'type': 'reference', 'reference_ids': [0]},                                                    
                   {                                                                                               
                       'type': 'text',                                                                             
                       'text': '.\n\n2. **Agent Systems**:\n   - **Applications**: LangGraph                       
           provides a framework for building agent-based systems, useful in various applications                   
           such as robotics, autonomous vehicles, and video games '                                                
                   },                                                                                              
                   {'type': 'reference', 'reference_ids': [0]},                                                    
                   {                                                                                               
                       'type': 'text',                                                                             
                       'text': '.\n\n3. **LLM Applications**:\n   - **Sophisticated AI Models**:                   
           Developers can use LangGraph’s capabilities to build more sophisticated AI models that                  
           learn and improve over time. For example, Norwegian Cruise Line uses LangGraph to                       
           create and refine guest-facing AI solutions, enhancing personalized guest experiences '                 
                   },                                                                                              
                   {'type': 'reference', 'reference_ids': [0]},                                                    
                   {                                                                                               
                       'type': 'text',                                                                             
                       'text': '.\n\n4. **Scalability**:\n   - **Effi

In [ ]:
user_input: str = "Yes, that's very helpful. Maybe I'll build an autonomous agent with it."
config = {"configurable": {"thread_id": "1"}}

events = graph.astream(
    {"messages": [{"role": "user", "content": user_input}]},
    config=config,
    stream_mode="values",
)
async for event in events:
    if "messages" in event:
        # event["messages"][-1].pretty_print()
        console.log(event["messages"][-1])

[15:10:07] HumanMessage(                                                                           ]8;id=39996;file:///var/folders/ny/dl75sc_x2tb54lsymt5bh5p00000gn/T/ipykernel_37674/1630387985.py\1630387985.py]8;;\:]8;id=992759;file:///var/folders/ny/dl75sc_x2tb54lsymt5bh5p00000gn/T/ipykernel_37674/1630387985.py#12\12]8;;\
               content="Yes, that's very helpful. Maybe I'll build an autonomous agent with it.",                  
               additional_kwargs={},                                                                               
               response_metadata={},                                                                               
               id='b61bbd80-81fd-4e7c-9e56-a342a3740ae9'                                                           
           )                                                                                                       

[15:10:17] AIMessage(                                                                              ]8;id=858515;file:///var/folders/ny/dl75sc_x2tb54lsymt5bh5p00000gn/T/ipykernel_37674/1630387985.py\1630387985.py]8;;\:]8;id=184375;file:///var/folders/ny/dl75sc_x2tb54lsymt5bh5p00000gn/T/ipykernel_37674/1630387985.py#12\12]8;;\
               content="That sounds like an exciting project! Building an autonomous agent with                    
           LangGraph would allow you to leverage its advanced capabilities in managing complex                     
           workflows and decision-making processes. Here are a few tips to get you started:\n\n1.                  
           **Understand the Requirements**:\n   - **Define the Scope**: Clearly outline what you                   
           want your autonomous agent to do. Whether it's for a specific task like navigation,                     
           data analysis, or customer interaction, having a clear scope will guide your                            
           development process.\n\n2. **Leverage LangGraph’s Features**:\n   - **Graph-Based                       
           Architecture**: Utilize LangGraph’s graph-based architecture to model the relationships                 
           and dependencies within your agent’s workflow. This will help in making more informed                   
           decisions based on past actions and feedback.\n   - **Stateful Orchestration**: Ensure                  
           your agent can maintain state and context over time, which is crucial for tasks that                    
           require memory of past interactions or decisions.\n\n3. **Development and                               
           Deployment**:\n   - **Use SDKs**: LangGraph supports SDKs in Python and JavaScript, so                  
           choose the one you are most comfortable with.\n   - **LangGraph Platform**: Consider                    
           using the LangGraph Platform for deploying and scaling your agent. The platform                         
           provides tools for building agent UXs and an integrated developer studio, which can                     
           streamline your development process.\n\n4. **Testing and Optimization**:\n   -                          
           **Iterative Improvement**: Continuously test and optimize your agent. Use real-world                    
           data to refine its decision-making processes and improve its performance over time.\n                   
           - **Feedback Loop**: Implement a feedback loop where the agent can learn from its                       
           mistakes and improve, making it more robust and reliable.\n\n5. **Scalability**:\n   -                  
           **Efficient Workflows**: Ensure that your agent can scale efficiently. LangGraph’s                      
           architecture is designed to handle complex workflows without sacrificing performance,                   
           which is essential for real-world applications.",                                                       
               additional_kwargs={},                                                                               
               response_metadata={                                                                                 
                   'token_usage': {                                                                                
                       'prompt_tokens': 2482,                                                                      
                       'total_tokens': 2911,                                                                       
                       'completion_tokens': 429                                                                    
                   },                                                                                              
                   'model_name': 'mistral-large-latest',             

<br>

### Replay The Full State History

- You can `replay` the full state history to see everything that occurred in the graph.

In [31]:
to_replay = None

for state in graph.get_state_history(config):
    print(f"Num Messages: {len(state.values['messages'])}, Next: {state.next}")
    print("-" * 80)
    # Select an arbitrary state to replay
    if len(state.values["messages"]) == 6:
        to_replay = state

Num Messages: 4, Next: ()
--------------------------------------------------------------------------------
Num Messages: 3, Next: ('chatbot',)
--------------------------------------------------------------------------------
Num Messages: 6, Next: ()
--------------------------------------------------------------------------------
Num Messages: 5, Next: ('chatbot',)
--------------------------------------------------------------------------------
Num Messages: 4, Next: ('__start__',)
--------------------------------------------------------------------------------
Num Messages: 4, Next: ()
--------------------------------------------------------------------------------
Num Messages: 3, Next: ('chatbot',)
--------------------------------------------------------------------------------
Num Messages: 2, Next: ('tools',)
--------------------------------------------------------------------------------
Num Messages: 1, Next: ('chatbot',)
----------------------------------------------------------

<br>

### Resume From A Checkpoint

- Resuming from a checkpoint will call the next `action` node.

In [32]:
print(to_replay.next)
print(to_replay.config)

()
{'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f038a8d-1d4c-66e2-8006-24cbf3f0d6c9'}}


<br>

### Load A State From A Moment In Time

- The checkpoint's to_replay.config contains a checkpoint_id timestamp.
- This timestamp tells LangGraph's checkpointer to load the state from that moment in time.

In [33]:
async for event in graph.astream(None, to_replay.config, stream_mode="values"):
    if "messages" in event:
        # event["messages"][-1].pretty_print()
        console.log(event["messages"][-1].content)

[15:29:51] That sounds like an exciting project! Building an autonomous agent with LangGraph would  ]8;id=388191;file:///var/folders/ny/dl75sc_x2tb54lsymt5bh5p00000gn/T/ipykernel_37674/3279626890.py\3279626890.py]8;;\:]8;id=120653;file:///var/folders/ny/dl75sc_x2tb54lsymt5bh5p00000gn/T/ipykernel_37674/3279626890.py#4\4]8;;\
           allow you to leverage its advanced capabilities in managing complex workflows and                       
           decision-making processes. Here are a few tips to get you started:                                      
                                                                                                                   
           1. **Understand the Requirements**:                                                                     
              - **Define the Scope**: Clearly outline what you want your autonomous agent to do.                   
           Whether it's for a specific task like navigation, data analysis, or customer                            
           interaction, having a clear scope will guide your development process.                                  
                                                                                                                   
           2. **Leverage LangGraph’s Features**:                                                                   
              - **Graph-Based Architecture**: Utilize LangGraph’s graph-based architecture to model                
           the relationships and dependencies within your agent’s workflow. This will help in                      
           making more informed decisions based on past actions and feedback.                                      
              - **Stateful Orchestration**: Ensure your agent can maintain state and context over                  
           time, which is crucial for tasks that require memory of past interactions or decisions.                 
                                                                                                                   
           3. **Development and Deployment**:                                                                      
              - **Use SDKs**: LangGraph supports SDKs in Python and JavaScript, so choose the one                  
           you are most comfortable with.                                                                          
              - **LangGraph Platform**: Consider using the LangGraph Platform for deploying and                    
           scaling your agent. The platform provides tools for building agent UXs and an integrated                
           developer studio, which can streamline your development process.                                        
                                                                                                                   
           4. **Testing and Optimization**:                                                                        
              - **Iterative Improvement**: Continuously test and optimize your agent. Use                          
           real-world data to refine its decision-making processes and improve its performance over                
           time.                                                                                                   
              - **Feedback Loop**: Implement a feedback loop where the agent can learn from its                    
           mistakes and improve, making it more robust and reliable.                                               
                                                                                                                   
           5. **Scalability**:                                                                                     
              - **Efficient Workflows**: Ensure that your agent can scale efficiently. LangGraph’s                 
           architecture is designed to handle complex workflows withou